# Install and Import 

In [1]:
# !pip install tensorflow==2.15.0
# !pip install gym
# !pip install keras
# !pip install keras-rl2
# %pip install scikit-learn

In [2]:
# %pip install ipykernel
# %pip install --upgrade nbformat
# %pip install stable-baselines3[extra]
# %pip install gymnasium

In [3]:
import numpy as np
import pandas as pd
import random
import logging
import math
from model_config import Path
import os

In [4]:
import gymnasium as gym
from gym import Env
from gymnasium import spaces
from gymnasium.utils import seeding

In [5]:
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import SAC

In [6]:
import decimal
decimal.getcontext().prec = 28  # Increase precision

In [7]:
import plotly.express as px
import plotly.graph_objects as go

# Open and Preprocessing Data

In [8]:
model_num = 8
data_num = 1

In [9]:
# Загрузка данных
df = pd.read_csv(Path["dataset"](model_num, data_num))

In [10]:
zero_close_prices = df[df['Close_orig'] == 0]
print(f"Number of zero 'Close' prices after scaling: {len(zero_close_prices)}")

Number of zero 'Close' prices after scaling: 0


In [11]:
unique_values = df['Asset_ID_encoded'].unique()
unique_values

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25], dtype=int64)

In [12]:
value_counts = df['Asset_ID_encoded'].value_counts()
value_counts

Asset_ID_encoded
1     4388
23    4388
18    4388
0     4387
5     4387
16    4384
15    4384
6     4379
25    4372
19    4371
4     4367
14    4342
13    4334
20    4314
3     4313
8     4309
22    4219
9     4207
21    4093
24    3602
11    3589
10    3582
17    3120
7     2957
2     2931
12    1847
Name: count, dtype: int64

In [13]:
# Разделение данных временного ряда на тренировочную и тестовую выборки.
def train_test_split_time_series(df, train_size=0.8):
    split_index = int(len(df) * train_size)
    train_df = df.iloc[:split_index].reset_index(drop=True)
    test_df = df.iloc[split_index:].reset_index(drop=True)
    return train_df, test_df

In [14]:
# Выполнение разделения
train_df, test_df = train_test_split_time_series(df, train_size=0.8)
print(f"Training data shape: {train_df.shape}")
print(f"Testing data shape: {test_df.shape}")

Training data shape: (83163, 26)
Testing data shape: (20791, 26)


# Game Rule

- Изменили window_length тепер оно работает и выставили его на 48
- Кроме того, модели подают нормализованную Close, а также RSI теперь без нормализации
- нормализация RobustScaler

In [15]:
window_length = 48
nb_steps = 80000

In [16]:
class TradingEnv(gym.Env):
    """
    Environment for training an agent to trade on the exchange using a continuous action space.
    """
    metadata = {'render.modes': ['human']}

    def __init__(self, df):
        super(TradingEnv, self).__init__()

        # Save data and initialize parameters
        self.df = df.reset_index(drop=True)
        self.total_steps = len(self.df) - 1
        self.window_length = window_length

        # Find indices where a new asset starts
        self.asset_start_indices = self._find_asset_start_indices()
        print(self.asset_start_indices)

        # Define action space: Continuous action between -1 and 1
        self.action_space = spaces.Box(low=-1, high=1, shape=(1,), dtype=np.float32)

        # Define observation space
        num_features = len(self.df.columns) - 1 # Вычли Close_orig
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(self.window_length, num_features), dtype=np.float32)

        # Initialize trading parameters
        self.fee_cost = 0.001
        self.initial_balance = 1000  # Starting balance
        self.balance = self.initial_balance
        self.net_worth = self.initial_balance
        self.prev_net_worth = self.net_worth

        self.reward = 0
        self.current_step = self.window_length
        self.current_price = 0
        self.shares_held = 0
        self.total_shares_sold = 0
        self.total_sales_value = 0

        self.now_token = (self.df.loc[self.current_step, 'Asset_ID_encoded'] - 1)
        self.prev_token = self.now_token

        self.hist = {
            "current_step": [],
            'balance': [],
            'net_worth': [],
            'shares_held': [],
            "token": [],
            "current_price": [],
            "reward": [],
            "action": [],
            'total_shares_sold': [],
            'total_sales_value': [],
        }

        self.logger = logging.getLogger(__name__)
        self.logger.setLevel(logging.INFO)

        if self.logger.hasHandlers():
            self.logger.handlers.clear()

        log_file = Path["train_log"](model_num, data_num + 1)
        file_handler = logging.FileHandler(log_file)
        file_handler.setLevel(logging.INFO)

        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        file_handler.setFormatter(formatter)

        self.logger.addHandler(file_handler)
        logging.getLogger().handlers = []
        self.seed = 999
        self.set_seed()


    def _find_asset_start_indices(self):
        """
        Find the indices in the DataFrame where a new asset starts.
        """
        asset_ids = self.df['Asset_ID_encoded']
        start_indices = {0:0}
        for i in range(1, len(asset_ids)):
            if asset_ids[i] != asset_ids[i - 1]:
                start_indices[asset_ids[i]] = i

        return start_indices


    def reset(self, seed = None, options=None, reset_hist=False):
        super().reset(seed=self.seed)
        self.set_seed()
        self.logger.info("Environment reset")

        # Existing reset logic
        self.balance = self.initial_balance
        self.net_worth = self.initial_balance
        self.prev_net_worth = self.net_worth
        self.shares_held = 0
        self.total_shares_sold = 0
        self.total_sales_value = 0

        # Move to the next asset's starting index
        self.now_token += 1
        if self.now_token not in self.asset_start_indices:
            self.now_token = list(self.asset_start_indices.keys())[0]  # Loop back to the first asset

        self.current_step = self.asset_start_indices[self.now_token] + self.window_length
        self.now_token = self.df.loc[self.current_step, 'Asset_ID_encoded']
        self.prev_token = self.now_token

        self.logger.info(f"Starting new episode with token {self.now_token} at step {self.current_step}")

        # Reset hist only if reset_hist is True
        if reset_hist:
            self.hist = {
                "current_step": [],
                'balance': [],
                'net_worth': [],
                'shares_held': [],
                "token": [],
                "current_price": [],
                "reward": [],
                "action": [],
                'total_shares_sold': [],
                'total_sales_value': [],
            }

        observation = self._next_observation()
        info = {}
        return observation, info


    def _next_observation(self):
        frame = self.df.drop(columns = ['Close_orig']).loc[self.current_step - self.window_length + 1:self.current_step]
        # obs = np.concatenate([
        #     frame.values,
        #     [self.shares_held],
        #     [self.balance],
        #     [self.net_worth]
        # ])
        obs = frame.values
        return obs.astype(np.float32)


    def step(self, action):
        self.reward = 0
        if isinstance(action, (list, np.ndarray)):
            action = action[0]

        self.logger.info(f"Step: {self.current_step}, Action taken: {action}")
        self._take_action(action)

        self.prev_token = self.now_token

        terminated = False
        truncated = False

        self.current_step += 1  # Move to the next time step

        if self.current_step >= self.total_steps:
            terminated = True
        else:
            self.now_token = self.df.loc[self.current_step, 'Asset_ID_encoded']
            if self.now_token != self.prev_token:
                self.logger.info(f"Token change at step {self.current_step}: {self.prev_token} -> {self.now_token}")
                self._sell_all_tokens()

        if self.prev_net_worth != 0:
            net_worth_change = self.net_worth - self.prev_net_worth
            percent_change = (net_worth_change) / self.prev_net_worth * 100

            if percent_change > 0:
                self.reward += percent_change
            else:
                self.reward -= percent_change * 2  

            if self.net_worth < self.initial_balance * 0.5:
                self.reward += -50  
                self.logger.info("Net worth dropped below 50% of initial balance.")
                terminated = True
        else:
            self.logger.info("prev_net_worth == 0")

        self.prev_net_worth = self.net_worth

        obs = self._next_observation()
        info = {}

        self.logger.info(f"Net worth: {self.net_worth}, Balance: {self.balance}, Reward: {self.reward}")

        self.hist["current_step"].append(self.current_step)
        self.hist["balance"].append(self.balance)
        self.hist["net_worth"].append(self.net_worth)
        self.hist["shares_held"].append(self.shares_held)
        self.hist["token"].append(self.now_token)
        self.hist["current_price"].append(self.current_price)
        self.hist["reward"].append(self.reward)
        self.hist["action"].append(action)
        self.hist["total_shares_sold"].append(self.total_shares_sold)
        self.hist["total_sales_value"].append(self.total_sales_value)

        return obs, self.reward, terminated, truncated, info


    def _sell_all_tokens(self):
        """
        Sell all tokens held at the current price.
        """
        self.current_price = self.df.loc[self.current_step - 1, 'Close_orig']
        shares_to_sell = self.shares_held
        if shares_to_sell > 0:
            # Determine total sale amount
            total_sale = shares_to_sell * self.current_price
            transaction_cost = total_sale * self.fee_cost
            total_sale -= transaction_cost  # Corrected: subtract transaction cost

            self.balance += total_sale
            self.shares_held = 0
            self.total_shares_sold += shares_to_sell  # Corrected: use shares_to_sell
            self.total_sales_value += total_sale

            # Update net worth
            self.net_worth = self.balance

            self.logger.info(f"Sold all shares of token {self.prev_token} at price {self.current_price}")
            self.logger.info(f"Total sale: {total_sale}, Transaction cost: {transaction_cost}")
        else:
            self.logger.info("No shares to sell.")


    def _take_action(self, action):
        """
        Apply the continuous action to the current state.
        """
        self.current_price = self.df.loc[self.current_step, 'Close_orig']

        action = float(np.clip(action, -1, 1))

        if action < 0:
            proportion = -action  # Convert to positive
            shares_to_sell = int(self.shares_held * proportion)
            self._sell(shares_to_sell)

        elif action > 0:
            proportion = action
            self._buy(proportion)
        else:
            # Hold
            self.reward += -1
            pass  

        self.net_worth = self.balance + self.shares_held * self.current_price


    def _buy(self, proportion):
        amount_to_spend = self.balance * proportion

        shares_to_buy = int(amount_to_spend / (self.current_price * (1 + self.fee_cost)))

        if shares_to_buy > 0:
            total_cost = shares_to_buy * self.current_price
            transaction_cost = total_cost * self.fee_cost
            total_cost += transaction_cost

            self.balance -= total_cost
            self.shares_held += shares_to_buy

            self.logger.info(f"Bought {shares_to_buy} shares at price {self.current_price}")
            self.logger.info(f"Total cost: {total_cost}, Transaction cost: {transaction_cost}")
        else:
            self.reward += -5
            self.logger.info("Not enough balance to buy.")


    def _sell(self, shares_to_sell):
        if shares_to_sell > self.shares_held:
            shares_to_sell = self.shares_held  # Can't sell more than held

        if shares_to_sell > 0:
            total_sale = shares_to_sell * self.current_price
            transaction_cost = total_sale * self.fee_cost
            total_sale -= transaction_cost

            self.balance += total_sale
            self.shares_held -= shares_to_sell
            self.total_shares_sold += shares_to_sell
            self.total_sales_value += total_sale

            self.logger.info(f"Sold {shares_to_sell} shares at price {self.current_price}")
            self.logger.info(f"Total sale: {total_sale}, Transaction cost: {transaction_cost}")
        else:
            self.reward += -5
            self.logger.info("No shares to sell.")


    def set_seed(self):
        # self.np_random, seed = seeding.np_random(seed)
        np.random.seed(self.seed)
        random.seed(self.seed)
        return [self.seed]


    def render(self, mode='human', close=False):
        profit = self.net_worth - self.initial_balance
        print(f'Step: {self.current_step}')
        print(f'Balance: {self.balance:.2f}')
        print(f'Shares held: {self.shares_held}')
        print(f'Net worth: {self.net_worth:.2f}')
        print(f'Profit: {profit:.2f}')

In [17]:
train_env = TradingEnv(train_df)

{0: 0, 1: 4387, 2: 8775, 3: 11706, 4: 16019, 5: 20386, 6: 24773, 7: 29152, 8: 32109, 9: 36418, 10: 40625, 11: 44207, 12: 47796, 13: 49643, 14: 53977, 15: 58319, 16: 62703, 17: 67087, 18: 70207, 19: 74595, 20: 78966}


In [18]:
test_env = TradingEnv(test_df)

{0: 0, 21: 117, 22: 4210, 23: 8429, 24: 12817, 25: 16419}


In [19]:
# check_env(train_env)

# Train Model

## Settings

In [20]:
model = SAC(
    policy='MlpPolicy',  # Use a Multi-Layer Perceptron policy
    env=train_env,
    verbose=1,
    learning_rate=1e-4,  # Adjust learning rate if needed
    batch_size=512,      # Adjust batch size if needed
    tensorboard_log="./sac_tensorboard/"  # Directory for TensorBoard logs
)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [21]:
# Set up the evaluation callback
from stable_baselines3.common.callbacks import EvalCallback
eval_callback = EvalCallback(
    test_env,                         # Evaluation environment
    best_model_save_path= Path["model"](model_num),   # Directory to save the best model
    log_path= Path["train_log"](model_num, data_num),               # Directory to save evaluation logs
    eval_freq=5000,                   # Evaluate every 5000 steps
    n_eval_episodes=3,                # Number of episodes to evaluate
    deterministic=True,               # Use deterministic actions during evaluation
    render=False                      # Disable rendering during evaluation
)

## Train

In [22]:
model.learn(
    total_timesteps=nb_steps,
    log_interval=1000,        # Log every 1000 steps
    # callback=eval_callback    # Pass the evaluation callback here
)

Logging to ./sac_tensorboard/SAC_22


In [23]:
# Save the trained model
model.save(Path["model"](model_num))

In [24]:
train_hist = train_env.hist
print(len(train_hist["action"]))
train_hist_df = pd.DataFrame(train_hist)

80000


In [25]:
unique_values = train_hist_df['token'].unique()
unique_values

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20], dtype=int64)

# Visualisaton

## Functions

In [26]:
def plot_close_by_asset(df, asset_id):
    # Фильтрация данных по Asset_ID_encoded
    asset_data = df[df['Asset_ID_encoded'] == asset_id]
    
    # Построение графика Close к индексу DataFrame
    fig = px.line(asset_data, x=asset_data.index, y='Close_orig', 
                  title=f'Close Price for Asset ID {asset_id}', 
                  labels={'index': 'Index', 'Close': 'Close Price'})
    
    # Показать график
    fig.show()

In [27]:
def plot_price_change_by_asset(df, asset_id):
    # Фильтрация данных по Asset_ID_encoded
    asset_data = df[df['Asset_ID_encoded'] == asset_id].copy()
    
    # Вычисление процентного изменения цены (Close)
    asset_data['Price_Change_Percent'] = asset_data['Close_orig'].pct_change() * 100
    
    # Построение графика изменения цены в процентах
    fig = px.line(asset_data, x=asset_data.index, y='Price_Change_Percent', 
                  title=f'Price Change Percentage for Asset ID {asset_id}', 
                  labels={'index': 'Index', 'Price_Change_Percent': 'Price Change (%)'})
    
    # Показать график
    fig.show()

In [28]:
def plot_token_data(df, token):
    # Фильтрация данных по выбранному токену
    token_data = df[df['token'] == token]
    
    # Вычисление среднего значения net_worth для данного токена
    avg_net_worth = token_data['net_worth'].mean()

    # Создание графика
    fig = go.Figure()

    # Линия net_worth
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['net_worth'], mode='lines', name='Net Worth'))

    # Горизонтальная линия для net_worth = 1000
    fig.add_hline(y=1000, line_color="green", name='Net Worth = 1000')

    # Горизонтальная линия для среднего значения net_worth
    fig.add_hline(y=avg_net_worth, line_color="red", name=f'Average Net Worth = {avg_net_worth:.2f}')

    # Настройка заголовков и осей
    fig.update_layout(title=f'Net Worth and Average for {token}',
                      xaxis_title='Current Step',
                      yaxis_title='Net Worth')

    # Показать график
    fig.show()

In [29]:
def plot_action_counts(df, token):
    # Фильтрация данных по токену
    token_data = df[df['token'] == token]
    
    # Подсчет количества каждого уникального действия для данного токена
    action_counts = token_data['action'].value_counts().reset_index()
    action_counts.columns = ['action', 'count']

    # Построение бар-чарта для отображения количества каждого действия
    fig = px.bar(action_counts, x='action', y='count', title=f'Count of Actions for {token}', labels={'action': 'Action', 'count': 'Count'})

    # Показать график
    fig.show()

In [30]:
def plot_relative_change_by_token(df, token):
    # Фильтрация данных по токену
    token_data = df[df['token'] == token].copy()

    # Вычисление относительного изменения для current_price и net_worth
    token_data['Price_Change_Percent'] = token_data['current_price'].pct_change() * 100
    token_data['NetWorth_Change_Percent'] = token_data['net_worth'].pct_change() * 100

    # Создание графика
    fig = go.Figure()

    # Линия для изменения current_price
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['Price_Change_Percent'],
                             mode='lines', name='Current Price Change (%)'))

    # Линия для изменения net_worth
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['NetWorth_Change_Percent'],
                             mode='lines', name='Net Worth Change (%)'))

    # Настройка заголовков и осей
    fig.update_layout(title=f'Relative Change of Current Price and Net Worth for {token}',
                      xaxis_title='Current Step',
                      yaxis_title='Change (%)')

    # Показать график
    fig.show()

## Plots

In [31]:
token = 19
plot_token_data(df = train_hist_df, token = token)
plot_close_by_asset(df= train_df, asset_id= token)
plot_price_change_by_asset(df= train_df, asset_id= token)
plot_relative_change_by_token(df = train_hist_df, token = token)

In [32]:
train_hist_df

,current_step,balance,net_worth,shares_held,token,current_price,reward,action,total_shares_sold,total_sales_value
0,49,1000.000000,1000.000000,0,0,0.000057,-5.000000,-0.210511,0,0.000000
1,50,1000.000000,1000.000000,0,0,0.000059,-5.000000,-0.330231,0,0.000000
2,51,689.710076,999.690020,5315886,0,0.000058,0.061996,0.310290,0,0.000000
3,52,969.921688,993.062360,405138,0,0.000057,1.325943,-0.923787,4910748,280.211612
4,53,577.280704,992.398264,7354112,0,0.000056,0.133747,0.404817,4910748,280.211612
...,...,...,...,...,...,...,...,...,...,...
79995,32702,0.000115,1570.535563,8802463,8,0.000178,-4.620718,0.999456,239373691,192693.403580
79996,32703,0.000115,1563.731259,8802463,8,0.000178,-4.133505,0.983951,239373691,192693.403580
79997,32704,0.000115,1551.583860,8802463,8,0.000176,-3.446357,0.971157,239373691,192693.403580
79998,32705,0.000115,1557.419893,8802463,8,0.000177,-4.623866,0.907977,239373691,192693.403580


In [33]:
train_hist_df.describe()

,current_step,balance,net_worth,shares_held,token,current_price,reward,action,total_shares_sold,total_sales_value
count,80000.000000,8.000000e+04,8.000000e+04,8.000000e+04,80000.000000,8.000000e+04,80000.000000,80000.000000,8.000000e+04,8.000000e+04
mean,42104.800125,2.956527e+06,7.912492e+06,7.777905e+12,10.211063,5.368953e+00,-0.165876,0.169384,6.222818e+15,7.839565e+09
std,24074.939089,8.428649e+06,1.493559e+07,5.160473e+13,6.129140,1.597533e+01,3.801182,0.697626,1.284848e+16,1.092271e+10
min,49.000000,2.030087e-08,4.561013e+02,0.000000e+00,0.000000,9.000000e-09,-54.551183,-1.000000,0.000000e+00,0.000000e+00
25%,20736.000000,3.733230e+00,8.573755e+02,3.460000e+02,5.000000,1.684765e-04,-2.804645,-0.508382,3.801570e+05,8.877661e+04
50%,43162.500000,5.028423e+02,1.108051e+04,1.185051e+06,10.000000,1.730289e-01,0.148715,0.338963,4.049598e+10,2.817856e+06
75%,63162.250000,6.438241e+04,1.025495e+07,2.118708e+07,16.000000,2.394920e+00,1.001950,0.845988,5.977634e+12,1.799485e+10
max,83162.000000,1.005825e+08,1.321444e+08,7.450623e+14,20.000000,1.065761e+02,258.038779,1.000000,3.387214e+16,2.865958e+10


# Test Model

## Test

In [34]:
obs, info = test_env.reset(reset_hist=True)  # Reset hist at the beginning
for _ in range(len(test_df)):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = test_env.step(action)

    if terminated or truncated:
        obs, info = test_env.reset(reset_hist=False)  # Do not reset hist

In [35]:
test_hist = test_env.hist
test_hist_df = pd.DataFrame(test_hist)
print(len(test_hist["action"]))

20791


In [36]:
test_hist_df

,current_step,balance,net_worth,shares_held,token,current_price,reward,action,total_shares_sold,total_sales_value
0,49,38.520915,999.039481,117,20,8.209560,0.192104,0.969052,0,0.000000
1,50,5.558574,1001.673258,121,20,8.232353,0.263631,0.948455,0,0.000000
2,51,5.558574,1004.477805,121,20,8.255531,-4.720014,0.977201,0,0.000000
3,52,5.558574,1002.367255,121,20,8.238088,-4.579772,0.972918,0,0.000000
4,53,5.558574,1009.369495,121,20,8.295958,-4.301430,0.980426,0,0.000000
...,...,...,...,...,...,...,...,...,...,...
20786,1636,0.000002,829.074639,42861740,21,0.000019,-3.522092,0.994397,1590012715,39654.676778
20787,1637,0.000002,833.789430,42861740,21,0.000019,-4.431319,0.996209,1590012715,39654.676778
20788,1638,0.000002,831.989237,42861740,21,0.000019,-4.568190,0.994302,1590012715,39654.676778
20789,1639,0.000002,818.530651,42861740,21,0.000019,-1.764721,0.995152,1590012715,39654.676778


In [37]:
unique_values = test_hist_df['token'].unique()
unique_values

array([20, 21, 22, 23, 24, 25], dtype=int64)

## Plots

In [38]:
token = 25

In [39]:
plot_token_data(df = test_hist_df, token = token)

In [40]:
plot_close_by_asset(df= test_df, asset_id= token)

## Metrics

In [41]:
def evaluate_model(hist_df, test_df, initial_balance):
    """
    Evaluate the model's performance.

    Parameters:
    - full_hist: Dictionary containing the testing history.
    - test_df: DataFrame containing the test data.
    - initial_balance: Initial balance used in the environment.

    Returns:
    - report_df: DataFrame containing performance metrics per asset.
    - overall_metrics: Dictionary containing overall performance metrics.
    """

    # Ensure timestamps are in order
    hist_df = hist_df.sort_values('current_step').reset_index(drop=True)

    # List of assets
    assets = hist_df['token'].unique()

    # Initialize report DataFrame
    report = []

    for asset_id in assets:
        asset_hist = hist_df[hist_df['token'] == asset_id]
        asset_data = test_df[test_df['Asset_ID_encoded'] == asset_id]

        # Calculate total profit/loss
        final_net_worth = asset_hist['net_worth'].iloc[-1]
        total_profit = final_net_worth - initial_balance

        # Calculate ROI
        roi = (final_net_worth - initial_balance) / initial_balance * 100

        # Calculate Sharpe Ratio
        returns = asset_hist['net_worth'].pct_change().dropna()
        if returns.std() != 0:
            sharpe_ratio = (returns.mean() / returns.std()) * np.sqrt(252)  # Assuming daily data
        else:
            sharpe_ratio = np.nan  # Undefined if no variance

        # Calculate Maximum Drawdown
        cumulative_returns = (1 + returns).cumprod()
        cumulative_max = cumulative_returns.cummax()
        drawdown = (cumulative_returns - cumulative_max) / cumulative_max
        max_drawdown = drawdown.min()

        # Calculate Win Rate
        trades = asset_hist[asset_hist['action'] != 0]
        wins = trades[trades['net_worth'].diff() > 0]
        win_rate = len(wins) / len(trades) * 100 if len(trades) > 0 else np.nan

        # Buy-and-Hold Strategy
        initial_price = asset_data['Close_orig'].iloc[0]
        final_price = asset_data['Close_orig'].iloc[-1]
        buy_and_hold_profit = (final_price - initial_price) * (initial_balance / initial_price)
        buy_and_hold_roi = (final_price - initial_price) / initial_price * 100

        # Ideal Strategy
        min_price = asset_data['Close_orig'].min()
        max_price = asset_data['Close_orig'].max()
        ideal_profit = (max_price - min_price) * (initial_balance / min_price)
        ideal_roi = (max_price - min_price) / min_price * 100

        # Collect metrics
        report.append({
            'Asset_ID': asset_id,
            'Total Profit': total_profit,
            'ROI (%)': roi,
            'Sharpe Ratio': sharpe_ratio,
            'Max Drawdown (%)': max_drawdown * 100,
            'Win Rate (%)': win_rate,
            'Buy-and-Hold Profit': buy_and_hold_profit,
            'Buy-and-Hold ROI (%)': buy_and_hold_roi,
            'Ideal Profit': ideal_profit,
            'Ideal ROI (%)': ideal_roi,
            'Asset Price Change (%)': (final_price - initial_price) / initial_price * 100,
        })

    # Create DataFrame from report
    report_df = pd.DataFrame(report)

    # Calculate overall metrics
    overall_profit = report_df['Total Profit'].sum()
    overall_roi = (overall_profit / (initial_balance * len(assets))) * 100
    overall_sharpe = report_df['Sharpe Ratio'].mean()
    overall_win_rate = report_df['Win Rate (%)'].mean()
    overall_buy_and_hold_profit = report_df['Buy-and-Hold Profit'].sum()
    overall_buy_and_hold_roi = (overall_buy_and_hold_profit / (initial_balance * len(assets))) * 100

    overall_metrics = {
        'Total Profit': overall_profit,
        'ROI (%)': overall_roi,
        'Sharpe Ratio': overall_sharpe,
        'Win Rate (%)': overall_win_rate,
        'Buy-and-Hold Profit': overall_buy_and_hold_profit,
        'Buy-and-Hold ROI (%)': overall_buy_and_hold_roi,
    }

    return report_df, overall_metrics

In [42]:
# Example for one model
report_df, overall_metrics = evaluate_model(test_hist_df, test_df, initial_balance = 1000)

In [43]:
report_df

,Asset_ID,Total Profit,ROI (%),Sharpe Ratio,Max Drawdown (%),Win Rate (%),Buy-and-Hold Profit,Buy-and-Hold ROI (%),Ideal Profit,Ideal ROI (%),Asset Price Change (%)
0,20,35.388272,3.538827,0.720742,-6.220264,25.000000,-118.979657,-11.897966,230.293410,23.029341,-11.897966
1,21,-512.483805,-51.248381,-0.231184,-59.345948,29.347562,-279.352882,-27.935288,1596.459700,159.645970,-27.935288
2,22,3272.626924,327.262692,0.410845,-62.546196,45.251799,253802.131121,25380.213112,293446.958188,29344.695819,25380.213112
3,23,3303.406080,330.340608,0.218021,-87.020980,48.701003,-152.651940,-15.265194,9635.856272,963.585627,-15.265194
4,24,8038.810607,803.881061,0.335794,-58.957133,50.277624,594.470161,59.447016,2103.489180,210.348918,59.447016
5,25,7138.052640,713.805264,0.070807,-74.675175,49.245197,233.832793,23.383279,2790.670479,279.067048,23.383279


In [46]:
test_hist_df.describe()

,current_step,balance,net_worth,shares_held,token,current_price,reward,action,total_shares_sold,total_sales_value
count,20791.000000,2.079100e+04,20791.000000,2.079100e+04,20791.000000,20791.000000,20791.000000,20791.000000,2.079100e+04,2.079100e+04
mean,10200.821125,9.522036e+02,4728.241281,7.814273e+06,22.977009,78.068220,-2.204561,0.618296,2.025755e+08,4.835983e+05
std,6259.150317,2.463380e+03,3249.575649,1.619532e+07,1.437574,155.258504,3.305580,0.626410,5.159102e+08,4.546929e+05
min,49.000000,3.703505e-07,487.516195,0.000000e+00,20.000000,0.000011,-43.425190,-1.000000,0.000000e+00,0.000000e+00
25%,5197.500000,4.990202e-03,1799.671147,1.600000e+01,22.000000,0.008839,-4.576390,0.616119,1.476015e+07,4.654830e+04
50%,10395.000000,1.663351e+00,4142.729814,2.616800e+04,23.000000,0.275237,-3.481302,0.948853,1.532061e+07,3.643595e+05
75%,15592.500000,1.126299e+02,7162.897735,2.302860e+05,24.000000,8.341084,0.023420,0.986427,1.701632e+07,9.551492e+05
max,20790.000000,1.221501e+04,14762.034847,4.473094e+07,25.000000,674.820251,48.217666,1.000000,2.289594e+09,1.305339e+06


In [44]:
report_df.to_csv(os.path.join("models", f"model_v{model_num}", f"report_{model_num}_{data_num}.csv"))

In [45]:
overall_metrics

{'Total Profit': 21275.80071698914,
 'ROI (%)': 354.59667861648563,
 'Sharpe Ratio': 0.2541709800772863,
 'Win Rate (%)': 41.30386399986581,
 'Buy-and-Hold Profit': 254079.44959758603,
 'Buy-and-Hold ROI (%)': 4234.657493293101}